#### **LEVEL - 1**

In [3]:
import pandas as pd

train_df = pd.read_csv('UNSW_NB15_training-set.csv')
test_df = pd.read_csv('UNSW_NB15_testing-set.csv')

train_df.head(), test_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'UNSW_NB15_training-set.csv'

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

train = train_df.copy()
test = test_df.copy()

train = train.drop(columns=['id'])
test = test.drop(columns=['id'])

cat_cols = ['proto', 'service', 'state', 'attack_cat']

for col in cat_cols:
    le = LabelEncoder()
    all_vals = pd.concat([train[col], test[col]], axis=0)
    le.fit(all_vals)
    train[col] = le.transform(train[col])
    test[col] = le.transform(test[col])

X_train = train.drop(columns=['label'])
y_train = train['label']

X_test = test.drop(columns=['label'])
y_test = test['label']

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("Preprocessing done without unseen errors.")

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=15,
    batch_size=256,
    verbose=1
)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Predict
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# Basic accuracy
test_acc = (y_pred.flatten() == y_test).mean()
print(f"Test Accuracy: {test_acc:.4f}")

# F1 / Precision / Recall
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


In [ ]:
# Save model
model.save("netpulse_level1.h5")

# Save scaler
import pickle
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

print("Model + Scaler saved for Level 1.")

In [ ]:
# Pick one random sample from test
import numpy as np

idx = np.random.randint(0, len(X_test))
sample = X_test[idx:idx+1]

pred = model.predict(sample)[0][0]

label = "ATTACK" if pred > 0.5 else "NORMAL"

print(f"Sample #{idx} → {label}")

In [ ]:
for i in range(10):
    idx = np.random.randint(0, len(X_test))
    sample = X_test[idx:idx+1]
    pred = model.predict(sample)[0][0]
    label = "ATTACK" if pred > 0.5 else "NORMAL"
    print(f"{i+1}. #{idx} → {label}")

In [ ]:
print("=== NET PULSE — LEVEL 1 SUMMARY ===")
print("Dataset     : UNSW-NB15")
print("Mode        : Binary Intrusion Detection (NORMAL vs ATTACK)")
print("Model       : Feedforward Neural Network (Dense 64 → 32 → 1)")
print("Labels      : Normal (0), Attack (1)")
print("Train Size  :", len(X_train))
print("Test Size   :", len(X_test))
print("Test Acc    : {:.4f}".format(test_acc))
print("Status      : LEVEL 1 COMPLETE ✔")
print("Artifacts   : netpulse_level1.h5 + scaler.pkl")
print("Ready For   : Level 2 (Flow sequencing + CNN+LSTM + Multi-class + UI)")
print("====================================")

# **LEVEL - 2**

In [ ]:
# Make a copy to avoid mutation
train2 = train_df.copy()
test2 = test_df.copy()

# Define simplified mapping
low_suspicious = ['Reconnaissance', 'Fuzzers', 'Analysis']
high_malicious = ['DoS', 'Exploits', 'Shellcode', 'Worms', 'Generic', 'Backdoor']

def map_attack(cat):
    if cat == 'Normal':
        return 0
    elif cat in low_suspicious:
        return 1
    elif cat in high_malicious:
        return 2
    else:
        return 2  # unknown -> treat as high severity for safety

train2['class3'] = train2['attack_cat'].apply(map_attack)
test2['class3'] = test2['attack_cat'].apply(map_attack)

# Quick sanity check
print(train2['class3'].value_counts())
print(test2['class3'].value_counts())


In [ ]:
from sklearn.preprocessing import StandardScaler

# Drop id
train3 = train2.drop(columns=['id'])
test3 = test2.drop(columns=['id'])

# Same categorical columns
cat_cols = ['proto', 'service', 'state']

# Encode categories (train + test combo)
from sklearn.preprocessing import LabelEncoder

for col in cat_cols:
    le = LabelEncoder()
    all_vals = pd.concat([train3[col], test3[col]], axis=0)
    le.fit(all_vals)
    train3[col] = le.transform(train3[col])
    test3[col] = le.transform(test3[col])

# Features & labels
X_train3 = train3.drop(columns=['label', 'attack_cat', 'class3'])
y_train3 = train3['class3']

X_test3 = test3.drop(columns=['label', 'attack_cat', 'class3'])
y_test3 = test3['class3']

# Scale
scaler3 = StandardScaler()
X_train3 = scaler3.fit_transform(X_train3)
X_test3 = scaler3.transform(X_test3)

print("Multiclass preprocessing done.")

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# One-hot encode labels
y_train3_cat = to_categorical(y_train3, num_classes=3)
y_test3_cat = to_categorical(y_test3, num_classes=3)

# Baseline model
mc_model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train3.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])

mc_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

mc_model.summary()

In [ ]:
mc_history = mc_model.fit(
    X_train3, y_train3_cat,
    validation_split=0.2,
    epochs=15,
    batch_size=256,
    verbose=1
)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

pred_mc = mc_model.predict(X_test3)
pred_mc = pred_mc.argmax(axis=1)

print("Test Accuracy:", (pred_mc == y_test3).mean())
print("\nReport:\n", classification_report(y_test3, pred_mc))
print("\nConfusion Matrix:\n", confusion_matrix(y_test3, pred_mc))

In [ ]:
import numpy as np

SEQ_LEN = 50

def flow_to_sequence(row):
    spkts = int(row['spkts'])
    dpkts = int(row['dpkts'])
    dur = float(row['dur']) if row['dur'] > 0 else 0.001  # prevent div by zero

    # sizes
    sbytes = row['sbytes']
    dbytes = row['dbytes']

    # packet size approx
    if spkts > 0:
        src_sizes = [sbytes/spkts]*spkts
    else:
        src_sizes = []

    if dpkts > 0:
        dst_sizes = [dbytes/dpkts]*dpkts
    else:
        dst_sizes = []

    # timestamps (uniform distribution for simplicity)
    src_times = np.linspace(0, dur, len(src_sizes)) if len(src_sizes)>0 else []
    dst_times = np.linspace(0, dur, len(dst_sizes)) if len(dst_sizes)>0 else []

    # direction tagging
    src_dir = [+1]*len(src_sizes)
    dst_dir = [-1]*len(dst_sizes)

    # merge in time order
    packets = list(zip(src_times, src_sizes, src_dir)) + \
              list(zip(dst_times, dst_sizes, dst_dir))

    packets.sort(key=lambda x: x[0])  # sort by time

    # time gaps
    seq = []
    prev_t = 0
    for t, size, d in packets:
        gap = t - prev_t
        prev_t = t
        seq.append([size, gap, d])

    # pad / truncate to SEQ_LEN
    if len(seq) < SEQ_LEN:
        pad = [[0,0,0]]*(SEQ_LEN - len(seq))
        seq = seq + pad
    else:
        seq = seq[:SEQ_LEN]

    return seq


In [ ]:
# IMPORTANT: use train3 & test3 from multi-class preprocessing

train_sequences = np.array([flow_to_sequence(row) for _, row in train3.iterrows()])
test_sequences = np.array([flow_to_sequence(row) for _, row in test3.iterrows()])

print("Sequence shapes:")
print(train_sequences.shape, test_sequences.shape)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Dropout, BatchNormalization

# Build CNN + LSTM Hybrid
cnn_lstm = Sequential([
    Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(SEQ_LEN, 3)),
    BatchNormalization(),
    LSTM(32, return_sequences=False),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(3, activation='softmax')
])

cnn_lstm.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

cnn_lstm.summary()

In [ ]:
hist_seq = cnn_lstm.fit(
    train_sequences, y_train3,
    validation_split=0.15,
    epochs=10,
    batch_size=256,
    verbose=1
)

In [ ]:
pred_seq = cnn_lstm.predict(test_sequences)
pred_seq = pred_seq.argmax(axis=1)

from sklearn.metrics import classification_report, confusion_matrix

print("Test Accuracy:", (pred_seq == y_test3).mean())
print("\nReport:\n", classification_report(y_test3, pred_seq))
print("\nConfusion Matrix:\n", confusion_matrix(y_test3, pred_seq))


In [ ]:
from tensorflow.keras.layers import Input, Dense, Conv1D, LSTM, BatchNormalization, Dropout, Concatenate
from tensorflow.keras.models import Model

# Input A: Flow stats
flow_input = Input(shape=(X_train3.shape[1],))
flow_branch = Dense(32, activation='relu')(flow_input)
flow_branch = BatchNormalization()(flow_branch)

# Input B: Sequence
seq_input = Input(shape=(SEQ_LEN, 3))
seq_branch = Conv1D(32, kernel_size=3, activation='relu')(seq_input)
seq_branch = BatchNormalization()(seq_branch)
seq_branch = LSTM(32)(seq_branch)

# Merge branches
merged = Concatenate()([flow_branch, seq_branch])
merged = Dense(32, activation='relu')(merged)
merged = Dropout(0.2)(merged)
output = Dense(3, activation='softmax')(merged)

fusion_model = Model(inputs=[flow_input, seq_input], outputs=output)

fusion_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

fusion_model.summary()

In [ ]:
hist_fusion = fusion_model.fit(
    [X_train3, train_sequences],
    y_train3,
    validation_split=0.15,
    epochs=10,
    batch_size=256,
    verbose=1
)

In [ ]:
pred_fusion = fusion_model.predict([X_test3, test_sequences])
pred_fusion = pred_fusion.argmax(axis=1)

from sklearn.metrics import classification_report, confusion_matrix

print("Test Accuracy:", (pred_fusion == y_test3).mean())
print("\nReport:\n", classification_report(y_test3, pred_fusion))
print("\nConfusion Matrix:\n", confusion_matrix(y_test3, pred_fusion))

In [ ]:
# Save fusion model
fusion_model.save("netpulse_fusion_v2.h5")
print("Fusion model saved as netpulse_fusion_v2.h5")

In [ ]:
import pickle

with open("scaler3.pkl", "wb") as f:
    pickle.dump(scaler3, f)

print("Flow scaler saved as scaler3.pkl")

In [ ]:
class_map = {0: "Normal", 1: "Suspicious", 2: "Malicious"}

with open("class_map.pkl", "wb") as f:
    pickle.dump(class_map, f)

print("Class map saved as class_map.pkl")

In [ ]:
config = {
    "SEQ_LEN": SEQ_LEN,
    "version": "v2_fusion",
    "features": ["size", "time_gap", "direction"],
}

with open("config.pkl","wb") as f:
    pickle.dump(config, f)

print("Config saved as config.pkl")

# **APP**

In [5]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!sudo dpkg -i cloudflared-linux-amd64.deb

--2026-02-18 17:28:46--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2026.2.0/cloudflared-linux-amd64.deb [following]
--2026-02-18 17:28:46--  https://github.com/cloudflare/cloudflared/releases/download/2026.2.0/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/4fddf4d7-e02d-44dc-9e5a-ef9e28afdd54?sp=r&sv=2018-11-09&sr=b&spr=https&se=2026-02-18T18%3A06%3A07Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64.deb&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&

In [4]:
!streamlit run app.py --server.port 8501 --server.address 0.0.0.0 & sleep 3 && cloudflared tunnel --url http://localhost:8501

/bin/bash: line 1: streamlit: command not found
/bin/bash: line 1: cloudflared: command not found
